In [ ]:
import pandas as pd
import numpy as np

# BODY25 → COCO17 매핑 (OpenPose 기준)
MAPPING_BODY25_TO_COCO17 = [0,16,15,18,17,5,2,6,3,7,4,12,9,13,10,14,11]

def csv_to_mmaction2_sample(csv_path, frame_dir, label, img_shape=(1080,1920)):
    """
    예시 csv(헤더 포함, BODY25, [x, y, c] 순서) → MMACTION2 PoseDataset 샘플(dict)
    """
    df = pd.read_csv(csv_path)
    T = len(df)
    V25 = 25
    M, C = 1, 2  # 단일 사람, x/y 좌표

    # 1. BODY25 순서로 데이터 추출
    kp25 = np.zeros((M, T, V25, C), dtype=np.float32)
    score25 = np.zeros((M, T, V25), dtype=np.float32)
    for t, row in df.iterrows():
        for v in range(V25):
            x = row[v*3 + 0]
            y = row[v*3 + 1]
            c = row[v*3 + 2]
            kp25[0, t, v, 0] = x
            kp25[0, t, v, 1] = y
            score25[0, t, v] = c

    # 2. BODY25 → COCO17 변환
    kp17 = kp25[:, :, MAPPING_BODY25_TO_COCO17, :]
    score17 = score25[:, :, MAPPING_BODY25_TO_COCO17]

    # 3. MMACTION2 샘플 dict 반환
    return {
        'frame_dir': frame_dir,
        'label': label,
        'img_shape': img_shape,
        'original_shape': img_shape,
        'total_frames': T,
        'keypoint': kp17,
        'keypoint_score': score17
    }

# 사용 예시
if __name__ == '__main__':
    sample = csv_to_mmaction2_sample(
        csv_path=r'd:\golfDataset\dataset\train\balanced_true\crop_keypoint\20201203_General_071_DOS_A_M40_MM_049_crop.csv',
        frame_dir='20201203_General_071_DOS_A_M40_MM_049_crop',
        label=1,
        img_shape=(1080,1920)
    )
    import pickle
    with open('sample.pkl', 'wb') as f:
        pickle.dump({'split': {'xsub_val': [sample['frame_dir']]}, 'annotations': [sample]}, f)
    print('[OK] sample.pkl 저장')